<a href="https://colab.research.google.com/github/sor820/tesis-stylegan-retina/blob/main/StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- CONFIGURACIÓN DE INFRAESTRUCTURA ---
from google.colab import drive
import os
import sys

# Montaje de Google Drive
drive.mount('/content/drive', force_remount=True)

# Descarga del repositorio de trabajo
%cd /content
if not os.path.exists('stylegan2-ada-pytorch'):
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch.git

# Instalación de dependencias críticas
%cd stylegan2-ada-pytorch
!pip install pyspng ninja imageio-ffmpeg==0.4.3

Mounted at /content/drive
/content
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 577, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 577 (delta 21), reused 17 (delta 17), pack-reused 546 (from 2)
Receiving objects: 100% (577/577), 8.43 MiB | 14.06 MiB/s, done.
Resolving deltas: 100% (329/329), done.
/content/stylegan2-ada-pytorch
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: imageio-ffmpeg
    Found existing installation: imageio-ffmpeg 0.6.0
    Uninstalling imageio-ffmpeg-0.6.0:
      Successfully uninstalled imageio-ffmpeg-0.6.0


In [ ]:
# --- MOTOR DE COMPATIBILIDAD ---
import torch
import torch.nn.functional as F
from torch_utils.ops import upfirdn2d, bias_act

def patched_upfirdn2d(x, f, up=1, down=1, padding=0, flip_filter=False, gain=1):
    if f is None: f = torch.ones([1, 1], device=x.device)
    f = f.unsqueeze(0).unsqueeze(0).to(x.dtype)
    batch, channels, h, w = x.shape
    x = x.reshape(batch * channels, 1, h, w)
    if up > 1: x = F.interpolate(x, scale_factor=up, mode='nearest')
    x = F.pad(x, [padding]*4)
    return F.conv2d(x, f, stride=down).reshape(batch, channels, -1, -1) * gain

def patched_bias_act(x, b=None, dim=1, gain=1, clamp=None, act='linear', alpha=None):
    if b is not None: x = x + b.reshape([-1 if i==dim else 1 for i in range(x.ndim)])
    if act == 'lrelu': x = F.leaky_relu(x, alpha or 0.2)
    elif act == 'relu': x = F.relu(x)
    return x * gain if clamp is None else x.clamp(-clamp, clamp) * gain

# Sobrescribir las funciones originales para evitar errores de compilación CUDA
upfirdn2d.upfirdn2d = patched_upfirdn2d
upfirdn2d._init = lambda: True
bias_act.bias_act = patched_bias_act
bias_act._init = lambda: True
os.environ['PYTORCH_JIT'] = '0'

print("✅ Motor de compatibilidad inyectado correctamente.")

✅ Motor de compatibilidad inyectado correctamente.


In [ ]:
# --- VARIABLES DE RUTA Y CONFIGURACIÓN ---
DATASET_PATH = "/content/drive/MyDrive/Recursos/Conjuntos_de_datos/dataset_526.zip"
RESUME_PKL   = "/content/drive/MyDrive/ModelosIA/Recursos/weights/stylegan2-ffhq-256x256.pkl"
OUTPUT_DIR   = "/content/drive/MyDrive/training-runs"

# Usamos tu último snapshot (240) como punto de partida
RESUME_PATH = "/content/drive/MyDrive/training-runs/00033-dataset_526-mirror-paper256-kimg5000-batch16-ada-resumecustom-freezed10/network-snapshot-000112.pkl"

# Verificación de archivos antes de iniciar
if os.path.exists(DATASET_PATH) and os.path.exists(RESUME_PKL):
    print("✅ Todos los archivos de origen localizados.")
else:
    print("⚠️ Advertencia: Algunos archivos no se encuentran en las rutas especificadas.")

✅ Todos los archivos de origen localizados.


In [ ]:
# --- BLOQUE 4: EJECUCIÓN DEL ENTRENAMIENTO OPTIMIZADO ---
# Configuración ajustada para dataset de 526 imágenes
# kimg=5000: Meta realista para evitar sobreentrenamiento (overfitting)
# snap=1: Máxima seguridad de guardado (aprovechando tus 2TB)

!python train.py \
    --outdir={OUTPUT_DIR} \
    --data={DATASET_PATH} \
    --resume={RESUME_PATH} \
    --gpus=1 \
    --cfg=paper256 \
    --batch=16 \
    --mirror=1 \
    --aug=ada \
    --snap=1 \
    --metrics=none \
    --kimg=5000 \
    --freezed=10

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
/content/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/content/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/stylegan2-ada-pytorch/torch_utils/custom_ops.py", line 111, in get_plugin
    module = importlib.import_module(module_name)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^

In [ ]:
!pip install opensimplex

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.0/268.0 kB 9.3 MB/s eta 0:00:00


In [ ]:
!python generate.py \
  --outdir=/content/drive/MyDrive/results-00001-000200-1 \
  --trunc=0.5 \
  --seeds=1-100 \
  --network=/content/drive/MyDrive/training-runs/00033-dataset_526-mirror-paper256-kimg5000-batch16-ada-resumecustom-freezed10/network-snapshot-000112.pkl

Se truncaron las últimas líneas 5000 del resultado de transmisión.
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/stylegan2-ada-pytorch/torch_utils/custom_ops.py", line 111, in get_plugin
    module = importlib.import_module(module_name)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1324, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'upfirdn2d_plugin'

  warnings.warn('Failed to build CUDA kern